# Chapter 16: Natural Language Processing with RNNs and Attention

Bab ini melanjutkan eksplorasi RNN, kali ini dengan fokus pada **Natural Language Processing (NLP)**, sebuah bidang yang bertujuan untuk memungkinkan komputer memahami dan menghasilkan bahasa manusia.

Kita akan membahas beberapa aplikasi dan arsitektur canggih:
* **Text Generation:** Menggunakan Character-RNN untuk menghasilkan teks yang mirip dengan karya Shakespeare.
* **Sentiment Analysis:** Mengklasifikasikan teks (misalnya, ulasan film) sebagai positif atau negatif.
* **Neural Machine Translation (NMT):** Membangun model **Encoder-Decoder** untuk menerjemahkan kalimat dari satu bahasa ke bahasa lain.
* **Attention Mechanisms:** Teknik revolusioner yang memungkinkan model untuk fokus pada bagian-bagian relevan dari input, mengatasi keterbatasan memori RNN.
* **The Transformer:** Arsitektur yang sangat sukses yang sepenuhnya bergantung pada *attention*, tanpa menggunakan lapisan rekuren atau konvolusional.

## Generating Shakespearean Text Using a Character RNN

Salah satu aplikasi menarik dari RNN adalah menghasilkan teks. Dengan melatih sebuah model untuk memprediksi karakter berikutnya dalam sebuah kalimat, kita dapat menggunakannya untuk menghasilkan teks baru, karakter demi karakter.

Prosesnya melibatkan:
1. **Membuat Dataset:** Mengambil teks panjang (misalnya, semua karya Shakespeare) dan mengubahnya menjadi urutan ID karakter.
2. **Memotong Data:** Membagi urutan panjang menjadi banyak jendela teks yang lebih kecil (*windows*).
3. **Membangun Model:** Menggunakan lapisan RNN seperti GRU atau LSTM untuk memprediksi karakter berikutnya.
4. **Menghasilkan Teks:** Memberi model sebuah teks awal, lalu berulang kali memprediksi karakter berikutnya dan menambahkannya ke teks.

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Asumsikan max_id adalah jumlah karakter unik
max_id = 39 

# Membangun model Char-RNN
model_char_rnn = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                   dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                   dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

model_char_rnn.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
model_char_rnn.summary()

## Sentiment Analysis

Analisis sentimen adalah tugas klasifikasi teks, misalnya menentukan apakah sebuah ulasan film positif atau negatif. Alih-alih memproses karakter per karakter, kita biasanya memproses kata per kata.

Ini memerlukan dua langkah penting:
1. **Tokenization:** Memecah teks menjadi kata-kata atau sub-kata (*tokens*).
2. **Embedding:** Mengubah setiap token menjadi representasi vektor numerik yang padat. Lapisan `Embedding` di Keras sangat cocok untuk ini. Ia belajar representasi yang berguna untuk setiap kata selama pelatihan.

In [ ]:
import tensorflow_datasets as tfds
import numpy as np

# Memuat dataset IMDb reviews
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
train_set = datasets["train"].batch(32).prefetch(1)

# Konseptual: Model Analisis Sentimen
vocab_size = 10000
embed_size = 128

model_sentiment = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

model_sentiment.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# history = model_sentiment.fit(train_set, epochs=5) # Pelatihan akan membutuhkan prapemrosesan teks yang sesuai

## An Encoder–Decoder Network for Neural Machine Translation

Untuk tugas seperti terjemahan mesin, di mana panjang output bisa berbeda dari panjang input, arsitektur **Encoder–Decoder** sangat efektif.

* **Encoder:** Sebuah RNN yang membaca kalimat sumber dan mengubahnya menjadi satu representasi vektor (disebut *context vector* atau *thought vector*).
* **Decoder:** Sebuah RNN yang mengambil *context vector* dan menghasilkan terjemahan kata demi kata.

Masalah dengan arsitektur ini adalah *context vector* menjadi hambatan (*bottleneck*) informasi, terutama untuk kalimat yang panjang.

## Attention Mechanisms

**Attention Mechanism** adalah terobosan yang mengatasi masalah *bottleneck* pada model Encoder-Decoder. Ide utamanya adalah memungkinkan *decoder* untuk "memperhatikan" bagian-bagian yang berbeda dari kalimat sumber pada setiap langkah waktu saat menghasilkan output.

Ini tidak hanya meningkatkan kinerja secara dramatis pada kalimat panjang tetapi juga memberikan *interpretability*: kita bisa memvisualisasikan bagian mana dari input yang menjadi fokus model saat menghasilkan output tertentu.

## The Transformer

Pada tahun 2017, sebuah makalah berjudul "Attention Is All You Need" memperkenalkan arsitektur **Transformer**, yang sepenuhnya meninggalkan lapisan rekuren dan hanya mengandalkan *attention mechanism* (khususnya **self-attention**).

Komponen utamanya adalah **Multi-Head Attention**, yang memungkinkan model untuk secara bersamaan memperhatikan informasi dari *subspace* representasi yang berbeda. Karena tidak ada rekurensi, Transformer menambahkan **Positional Embeddings** ke input untuk memberikan informasi tentang urutan kata.

Arsitektur ini lebih mudah diparalelkan daripada RNN dan telah menjadi dasar bagi model bahasa canggih seperti BERT dan GPT.

### Visualisasi Pelatihan (Konseptual)

In [ ]:
import pandas as pd

# Data dummy untuk history pelatihan
dummy_history = {
    'loss': [1.5, 0.8, 0.5, 0.3, 0.2],
    'accuracy': [0.5, 0.7, 0.8, 0.9, 0.95],
    'val_loss': [1.6, 0.9, 0.6, 0.4, 0.3],
    'val_accuracy': [0.48, 0.68, 0.78, 0.88, 0.92]
}

pd.DataFrame(dummy_history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 2)
plt.title("Contoh Kurva Pembelajaran untuk Analisis Sentimen")
plt.xlabel('Epoch')
plt.ylabel('Loss / Accuracy')
plt.show()